In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
0.将 预测结果保持小数形式，而不是转换成整数形式  结果从0.5以上优化到0.5以下 0.47381812389478345
1.将 survey_time数据提取出月份、日期、小时三个特征 0.47143198583365375
2.将 family_income，income离散成5段的离散值 0.4730428161760329
3.将 birth 转换成年龄值age 0.47143198583365375
4.将 family_income，income离散成5段的离散值，并且birth 转换成年龄值age，都加上，0.47241532744595904
6.将 age 离散化成7段年龄段  0.47619062169156706
7.将 survey_time数据提取出一周的第几天、是否周末 两个特征 0.47097871771026456
8.将 family_income、income中小于0的数字转换为0 0.4708401593321734
9.使用全量特征训练 0.466104881012437
10.将 family_income，income提取出来家庭收入占个人收入的比例特征，0.46458776964941884
"""

import xgboost as xgb
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime

In [2]:
data_dir = '/Users/changqingli/pycharm_projects/algorithms/weekend_content/'
train_data_file = 'data/happiness_train_complete.csv'
test_data_file = 'data/happiness_test_complete.csv'


In [3]:
df_train = pd.read_csv(data_dir+train_data_file, encoding='gb2312')
df_test = pd.read_csv(data_dir+test_data_file, encoding='gb2312')
df_train.head()

,id,happiness,survey_type,province,city,county,survey_time,gender,birth,nationality,...,neighbor_familiarity,public_service_1,public_service_2,public_service_3,public_service_4,public_service_5,public_service_6,public_service_7,public_service_8,public_service_9
0,1,4,1,12,32,59,2015/8/4 14:18,1,1959,1,...,4,50,60,50,50,30.0,30,50,50,50
1,2,4,2,18,52,85,2015/7/21 15:04,1,1992,1,...,3,90,70,70,80,85.0,70,90,60,60
2,3,4,2,29,83,126,2015/7/21 13:24,2,1967,1,...,4,90,80,75,79,80.0,90,90,90,75
3,4,5,2,10,28,51,2015/7/25 17:33,2,1943,1,...,3,100,90,70,80,80.0,90,90,80,80
4,5,4,1,7,18,36,2015/8/10 9:50,2,1994,1,...,2,50,50,50,50,50.0,50,50,50,50


In [4]:
df_train = df_train.drop('id', 1)
df_train = df_train[df_train['happiness'] != -8]
df_train = df_train[pd.to_datetime(df_train.survey_time).dt.month != 2]


In [5]:
# survey_time数据提取出月份、日期、小时三个特征
df_train['month'] = pd.to_datetime(df_train.survey_time).dt.month
df_train['day'] = pd.to_datetime(df_train.survey_time).dt.day
df_train['hour'] = pd.to_datetime(df_train.survey_time).dt.hour
df_train['dayofweek'] = pd.to_datetime(df_train.survey_time).dt.dayofweek+1
df_train['is_weekend'] = (pd.to_datetime(df_train.survey_time).dt.dayofweek+1).apply(lambda x: 1 if x in (6, 7) else 0)
df_test['month'] = pd.to_datetime(df_test.survey_time).dt.month
df_test['day'] = pd.to_datetime(df_test.survey_time).dt.day
df_test['hour'] = pd.to_datetime(df_test.survey_time).dt.hour
df_test['dayofweek'] = pd.to_datetime(df_test.survey_time).dt.dayofweek+1
df_test['is_weekend'] = (pd.to_datetime(df_test.survey_time).dt.dayofweek+1).apply(lambda x: 1 if x in (6, 7) else 0)
# 去除 survey_time数据
df_train = df_train.drop('survey_time', 1)
df_test = df_test.drop('survey_time', 1)
# 家庭收入是个人收入的比例
df_train['income_rate'] = df_train.apply(lambda row: row['family_income']/row['income'] if row['family_income']*row['income']>0 else 0, 1)
df_test['income_rate'] = df_test.apply(lambda row: row['family_income']/row['income'] if row['family_income']*row['income']>0 else 0, 1)
# 将family_income、income小于0的值转换为0值
df_train['family_income'] = df_train['family_income'].apply(lambda x: x if x>0 else 0)
df_train['income'] = df_train['income'].apply(lambda x: x if x>0 else 0)
df_test['family_income'] = df_test['family_income'].apply(lambda x: x if x>0 else 0)
df_test['income'] = df_test['income'].apply(lambda x: x if x>0 else 0)
# family_income，income离散成5段的离散值
family_income_quantile = df_train.family_income.quantile([0, 0.2, 0.4, 0.6, 0.8, 1]).values
df_train['family_income_stage'] = pd.cut(df_train.family_income, family_income_quantile, labels=[0, 1, 2, 3, 4])
income_quantile = df_train.income.quantile([0, 0.33, 0.66, 1]).values
df_train['income_stage'] = pd.cut(df_train.income, income_quantile, labels=[0, 1, 2])
family_income_quantile = df_test.family_income.quantile([0, 0.2, 0.4, 0.6, 0.8, 1]).values
df_test['family_income_stage'] = pd.cut(df_test.family_income, family_income_quantile, labels=[0, 1, 2, 3, 4])
income_quantile = df_test.income.quantile([0, 0.33, 0.66, 1]).values
df_test['income_stage'] = pd.cut(df_test.income, income_quantile, labels=[0, 1, 2])
# birth 转换成年龄值
df_train['age']  = (2019 - df_train.birth)
df_train = df_train.drop('birth', 1)
# df_train['age'] = pd.cut(df_train.age, [0, 20, 30, 40, 50, 60, 70, 100], labels=[0, 1, 2, 3, 4, 5, 6])
df_test['age']  = (2019 - df_test.birth)
df_test = df_test.drop('birth', 1)
# df_test['age'] = pd.cut(df_test.age, [0, 20, 30, 40, 50, 60, 70, 100], labels=[0, 1, 2, 3, 4, 5, 6])


df_train.head()


,happiness,survey_type,province,city,county,gender,nationality,religion,religion_freq,edu,...,public_service_9,month,day,hour,dayofweek,is_weekend,income_rate,family_income_stage,income_stage,age
0,4,1,12,32,59,1,1,1,1,11,...,50,8,4,14,2,0,3.000000,3,1,60
1,4,2,18,52,85,1,1,1,1,12,...,60,7,21,15,2,0,2.000000,2,1,27
2,4,2,29,83,126,2,1,0,3,4,...,75,7,21,13,2,0,4.000000,0,0,52
3,5,2,10,28,51,2,1,1,1,3,...,80,7,25,17,6,1,1.869159,1,1,76
4,4,1,7,18,36,2,1,1,1,12,...,50,8,10,9,1,0,2.000000,NaN,NaN,25


In [6]:
df_train = df_train.drop('property_other', 1).drop('edu_other', 1).drop('invest_other', 1)
df_test = df_test.drop('property_other', 1).drop('edu_other', 1).drop('invest_other', 1)

In [7]:
# df_train.loc[1, :]
# df_train['property_other'].value_counts()





In [8]:
# df_train.floor_area.value_counts()

In [9]:
len(df_train)
len(df_test)

2968

In [10]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
df_train_train = df_train.sample(frac=0.8, random_state=0, axis=0)
df_train_validation = df_train[~df_train.index.isin(df_train_train.index)]


In [11]:
len(df_train_validation)

1597

In [12]:
tr_data = df_train_train.as_matrix()
va_data = df_train_validation.as_matrix()
te_data = df_test.as_matrix()
tr_data

/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


array([[3, 1, 1, ..., 1, 0, 23],
       [4, 1, 12, ..., 2, 2, 24],
       [5, 1, 21, ..., 2, 2, 39],
       ...,
       [3, 2, 29, ..., 0, 0, 65],
       [5, 1, 1, ..., 0, 0, 47],
       [4, 2, 18, ..., 4, 0, 30]], dtype=object)

In [13]:
tr_x_data = tr_data[:, 1:]
tr_y_data = tr_data[:, 0]

va_x_data = va_data[:, 1:]
va_y_data = va_data[:, 0]

In [14]:
len(tr_x_data[0])
# df_train.edu_other.value_counts()

142

In [15]:
# (tr_x_data.index('统筹'))
# for i in range(len(tr_x_data)):
#     for j in range(len(tr_x_data[i])):
#         if tr_x_data[i][j] == '统筹':
#             print(i, j)

# list(df_train.columns)[84]


In [16]:
mod = xgb.XGBRegressor(max_depth=6,
                       learning_rate=0.1,
                       n_estimators=100,
                       objective='reg:squarederror',
                       booster='gbtree',
                       gamma=5,
                       reg_lambda=10,
                       min_child_weight=1)

In [17]:
mod.fit(tr_x_data, tr_y_data)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=5,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=10, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [18]:
va_pred_data = mod.predict(va_x_data)
np.mean((va_pred_data - va_y_data)*(va_pred_data - va_y_data))

0.46458776964941884

In [19]:
pred_data = te_data[:, 1:]
pred_res = mod.predict(pred_data)

import csv
result = 'submission_908_xgb_tiaocan_4.csv'
with open(data_dir+result, 'w') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(['id', 'happiness'])
    for i in range(len(te_data)):
        writer.writerow([int(te_data[i][0]), pred_res[i]])